# Hourly Data

In [7]:
import pandas as pd
import lightgbm as lgb
import numpy as np
# suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [8]:
df = pd.read_parquet("data/wind/enriched.parquet")
# df = df[["production", "rt_plant_id", "forecast_dt"]]
df.head()

,epoch,production,rt_plant_id,forecast_dt,UGRD_80.m.above.ground.SW,UGRD_80.m.above.ground.NW,UGRD_80.m.above.ground.NE,UGRD_80.m.above.ground.SE,VGRD_80.m.above.ground.SW,VGRD_80.m.above.ground.NW,VGRD_80.m.above.ground.NE,VGRD_80.m.above.ground.SE
index,,,,,,,,,,,,
0,1514754000,0.0,1503,2018-01-01 00:00:00+03:00,-0.842773,0.067227,-0.782773,0.067227,1.08792,2.18792,0.997922,2.03792
1,1514764800,0.0,1503,2018-01-01 03:00:00+03:00,-0.469976,0.720024,-0.179976,0.730024,1.73287,2.92287,1.242870,2.56287
2,1514775600,0.4,1503,2018-01-01 06:00:00+03:00,-0.932734,0.387266,-0.372734,0.477266,2.52399,3.67399,1.713990,3.02399
3,1514786400,2.0,1503,2018-01-01 09:00:00+03:00,-1.308400,-0.028396,-0.458396,0.201604,3.69407,4.74407,2.504070,3.39407
4,1514797200,2.0,1503,2018-01-01 12:00:00+03:00,-0.004195,1.555810,0.825805,1.795810,4.59009,5.41009,2.890090,3.56009


### Missing hours in data

93 plant için data 2018-01-01den başlıyor, kalanlar şöyle:
- 2288  &emsp;  2018-03-20
- 2291  &emsp;  2018-04-13
- 2323  &emsp;  2018-09-24
- 2326  &emsp;  2018-09-28
- 2374  &emsp;  2018-11-13
- 2397  &emsp;  2019-03-14
- 2420  &emsp;  2019-06-28
- 2538  &emsp;  2020-03-28

1 saatlik data 3 plant hariç hepsinde data 2019-01-21'de başlıyor, üç plant'in öncesinde 3 saatlik dataları da yok zaten
- 2397  &emsp;  2019-03-14
- 2420  &emsp;  2019-06-28
- 2538  &emsp;  2020-03-28

3 saatlik dönemde (2019-01-21 öncesi) :
- 6 saat fark olan bir gün var: 2018-05-27 06:00'de 

1 saatlik dönemde (2019-01-21 sonrası) :
- 3 saat fark olan 2019-01-24 00:00 ve 2019-01-24 03:00
- 2 saat fark olan 2019-04-15 21:00, 2019-05-05 00:00, 2019-08-17 04:00, 2021-06-08 08:00 ve 2021-06-09 20:00 

datayı 2019-01-24ten sonrasına filtrelersek 3 plant sonradan dahil oluyor olacak, 5 saat için de missing data oluyor

In [ ]:
# df["dayofweek"] = df.forecast_dt.dt.dayofweek
# df["hour"] = df.forecast_dt.dt.hour
df.sort_values(["rt_plant_id", "forecast_dt"], inplace=True)

In [ ]:
df["shifted"] = df.groupby("rt_plant_id").forecast_dt.shift(1)
df["diff"] = (df.forecast_dt - df.shifted)
df

In [ ]:
df["diff"].value_counts()

In [ ]:
df.groupby("rt_plant_id").forecast_dt.min()[df.groupby("rt_plant_id").forecast_dt.min() != "2018-01-01 00:00:00+03:00"]

In [ ]:
df[df["diff"] == pd.Timedelta(hours=1)].groupby("rt_plant_id").forecast_dt.min().value_counts()
# 2397   2019-03-14
# 2420   2019-06-28
# 2538   2020-03-28
# remaining: 2019-01-21 

In [ ]:
# 2019-01-24 00:00 2019-01-24 03:00 içinde 3er saatlik boşluk var
# 2019-01-24 04:00 buradan kesersek sadece 1 saatlik boşluk oluyor, 5 farklı saatte
print(df.loc[df["forecast_dt"] < "2019-01-21 04:00:00+03:00", "diff"].value_counts())
print(df.loc[df["forecast_dt"] >= "2019-01-21 04:00:00+03:00", "diff"].value_counts())

In [ ]:
df_ = df[df["forecast_dt"] < "2019-01-21 00:00:00+03:00"]
# df_[df_["diff"] == pd.Timedelta(hours=2)].rt_plant_id.value_counts()
# her plant'te boşluk var
df_[df_["diff"] != pd.Timedelta(hours=3)].dropna().forecast_dt.value_counts()

In [ ]:
df_ = df[df["forecast_dt"] >= "2019-01-21 00:00:00+03:00"]
# df_[df_["diff"] == pd.Timedelta(hours=2)].rt_plant_id.value_counts()
# her plant'te boşluk var
df_[df_["diff"] == pd.Timedelta(hours=2)].dropna().forecast_dt.value_counts()

In [9]:
df = df[df["forecast_dt"] >= "2019-01-24 03:00:00+03:00"]
# df = df.drop(columns=["shifted", "diff"]).reset_index(drop=True)
df = df.reset_index(drop=True)
df

,epoch,production,rt_plant_id,forecast_dt,UGRD_80.m.above.ground.SW,UGRD_80.m.above.ground.NW,UGRD_80.m.above.ground.NE,UGRD_80.m.above.ground.SE,VGRD_80.m.above.ground.SW,VGRD_80.m.above.ground.NW,VGRD_80.m.above.ground.NE,VGRD_80.m.above.ground.SE
0,1548288000,10.0,1503,2019-01-24 03:00:00+03:00,-2.381990,-1.811990,-1.301990,-1.89199,17.66920,15.35920,15.56920,10.63920
1,1548291600,10.0,1503,2019-01-24 04:00:00+03:00,-3.177610,-2.647610,-2.297610,-3.02761,16.50070,14.29070,14.46070,10.24070
2,1548295200,9.0,1503,2019-01-24 05:00:00+03:00,-3.593300,-3.043300,-2.573300,-3.40330,15.63730,13.43730,13.17730,9.39729
3,1548298800,2.7,1503,2019-01-24 06:00:00+03:00,-4.027460,-3.627460,-2.697460,-3.66746,14.39660,13.14660,11.37660,8.86664
4,1548302400,1.3,1503,2019-01-24 07:00:00+03:00,-4.512970,-4.192970,-2.722970,-3.82297,13.09370,12.96370,9.64367,8.58367
...,...,...,...,...,...,...,...,...,...,...,...,...
2650409,1643299200,20.0,2374,2022-01-27 19:00:00+03:00,-0.014146,-0.894145,0.225854,-1.14415,-9.65097,-5.86097,-5.42097,-4.08097
2650410,1643302800,18.5,2374,2022-01-27 20:00:00+03:00,0.542844,-0.977156,-0.307156,-1.18716,-9.24466,-5.59466,-4.40466,-3.31466
2650411,1643306400,18.1,2374,2022-01-27 21:00:00+03:00,-0.170332,-1.980330,-0.790332,-1.39033,-9.34740,-5.69740,-3.84740,-3.20740
2650412,1643310000,13.7,2374,2022-01-27 22:00:00+03:00,-0.388333,-2.448330,-0.778333,-1.53833,-8.65053,-5.44053,-3.42053,-2.99053


### Outlier

In [ ]:
# rolling 60 günlük quantile üzerinden detect etmeye çalış outlier'ı
# yüksek olması gereken zamanlarda tirbün durabiliyor, cutoff gibi, bunu detect etmenin yolunu bulabiliriz, tree yaklaşımı ya da quantile regression iş yapabilir
#  

### Rolling Percentile

In [ ]:
percentile_95 = df.groupby("rt_plant_id").production.rolling(30 * 24).quantile(0.99)
percentile_95.name = "percentile_95"
percentile_95 = percentile_95.reset_index(drop=True)
percentile_05 = df.groupby("rt_plant_id").production.rolling(30 * 24).quantile(0.01)
percentile_05.name = "percentile_05"
percentile_05 = percentile_05.reset_index(drop=True)

df = pd.concat([df, percentile_95, percentile_05], axis=1)

In [ ]:
df[df["rt_plant_id"] == 672].set_index("forecast_dt")[["production", "percentile_95", "percentile_05"]].plot(figsize=(20, 10))

### Percentile

In [10]:
def detect_outlier(df, lower=0.025, upper=0.975):
    q_lower = df.groupby("rt_plant_id").production.quantile(lower)
    q_lower.name = "production_lower"
    q_upper = df.groupby("rt_plant_id").production.quantile(upper)
    q_upper.name = "production_upper"
    df = pd.merge(df, q_lower, on="rt_plant_id", how="left")
    df = pd.merge(df, q_upper, on="rt_plant_id", how="left")
    return df

def clip_outlier(df):
    assert "production_lower" in df.columns
    assert "production_upper" in df.columns
    df.loc[:, "is_outlier"] = (df.production < df.production_lower) | (df.production > df.production_upper)
    df["production_cleaned"] = np.clip(df.production, df.production_lower, df.production_upper)
    return df

In [11]:
df = detect_outlier(df, 0.01, 0.999)
df = clip_outlier(df)

In [12]:
df

,epoch,production,rt_plant_id,forecast_dt,UGRD_80.m.above.ground.SW,UGRD_80.m.above.ground.NW,UGRD_80.m.above.ground.NE,UGRD_80.m.above.ground.SE,VGRD_80.m.above.ground.SW,VGRD_80.m.above.ground.NW,VGRD_80.m.above.ground.NE,VGRD_80.m.above.ground.SE,production_lower,production_upper,is_outlier,production_cleaned
0,1548288000,10.0,1503,2019-01-24 03:00:00+03:00,-2.381990,-1.811990,-1.301990,-1.89199,17.66920,15.35920,15.56920,10.63920,0.0,10.0,False,10.0
1,1548291600,10.0,1503,2019-01-24 04:00:00+03:00,-3.177610,-2.647610,-2.297610,-3.02761,16.50070,14.29070,14.46070,10.24070,0.0,10.0,False,10.0
2,1548295200,9.0,1503,2019-01-24 05:00:00+03:00,-3.593300,-3.043300,-2.573300,-3.40330,15.63730,13.43730,13.17730,9.39729,0.0,10.0,False,9.0
3,1548298800,2.7,1503,2019-01-24 06:00:00+03:00,-4.027460,-3.627460,-2.697460,-3.66746,14.39660,13.14660,11.37660,8.86664,0.0,10.0,False,2.7
4,1548302400,1.3,1503,2019-01-24 07:00:00+03:00,-4.512970,-4.192970,-2.722970,-3.82297,13.09370,12.96370,9.64367,8.58367,0.0,10.0,False,1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2650409,1643299200,20.0,2374,2022-01-27 19:00:00+03:00,-0.014146,-0.894145,0.225854,-1.14415,-9.65097,-5.86097,-5.42097,-4.08097,0.0,20.0,False,20.0
2650410,1643302800,18.5,2374,2022-01-27 20:00:00+03:00,0.542844,-0.977156,-0.307156,-1.18716,-9.24466,-5.59466,-4.40466,-3.31466,0.0,20.0,False,18.5
2650411,1643306400,18.1,2374,2022-01-27 21:00:00+03:00,-0.170332,-1.980330,-0.790332,-1.39033,-9.34740,-5.69740,-3.84740,-3.20740,0.0,20.0,False,18.1
2650412,1643310000,13.7,2374,2022-01-27 22:00:00+03:00,-0.388333,-2.448330,-0.778333,-1.53833,-8.65053,-5.44053,-3.42053,-2.99053,0.0,20.0,False,13.7


In [13]:
df.to_parquet("data/wind/2019-01-24_outlier_removed.parquet")

In [ ]:
df.groupby("rt_plant_id").is_outlier.sum().hist()

In [ ]:
# azalan 2062 1504
# artan 1524 1523 1939 1944
# artıp azalan 1939 1781
# garip artan 969
# garip azalan 1878
# stabil 1787
# net outlier 1488 2050

# capacity & outlier karışık 1761

In [ ]:
PLANT_ID = 2050
a__ = df[df["rt_plant_id"] == PLANT_ID]
a__[["production", "production_lower", "production_upper"]].plot(figsize=(20, 10))
# a__.groupby(a__.forecast_dt.dt.strftime("%Y%m")).production.max().plot(figsize=(15,5))

In [ ]:
find_percentile_change(df, 2062)

In [ ]:
a = df[df["rt_plant_id"] == PLANT_ID]
a.groupby(a.forecast_dt.dt.date).production.max().plot(figsize=(15,5))


In [ ]:
a["new_col"] = a.forecast_dt.dt.year.astype(str) + (a.forecast_dt.dt.weekofyear // 4 + 1).astype(str).str.zfill(2)
a.groupby("new_col").production.max().plot(figsize=(15,5))


In [ ]:
a = df[df["rt_plant_id"] == PLANT_ID]
a["new_col"] = a.forecast_dt.dt.strftime("%Y") + (a.forecast_dt.dt.strftime("%W").astype(int) // 6 + 1).astype(str).str.zfill(2)
a.groupby("new_col").production.max().plot(figsize=(15,5))


In [ ]:
df[df["rt_plant_id"] == PLANT_ID].set_index("forecast_dt")[["production", "production_lower", "production_upper"]].plot(figsize=(20, 10))

In [ ]:
m_ = df[df["rt_plant_id"] == PLANT_ID]
x = m_.production.rolling(10 * 24, closed="right")
m_["outlier1"] = x.mean() + 3 * x.std()
# a_.loc[a_["production"] > a_["outlier1"], "production"] = a_["outlier1"]
# m_["outlier2"] = x.mean() - 2 * x.std()
# a_.loc[a_["production"] < a_["outlier2"], "production"] = a_["outlier2"]
# a_.drop(["outlier1", "outlier2"], axis=1, inplace=True)
m_[(m_["forecast_dt"] > "2020-07-01") & (m_["forecast_dt"] < "2021-05-01")].set_index("forecast_dt")[["production", "production_lower", "production_upper", "outlier1"]].plot(figsize=(20, 10))


In [ ]:
plants = df.rt_plant_id.unique()

In [ ]:
PLANT_ID = plants[10]
df[df["rt_plant_id"] == PLANT_ID].set_index("forecast_dt")[["production", "production_lower", "production_upper"]].plot(figsize=(20, 10))

In [ ]:
def _remove_local_outliers(df, plant_id):
    a_ = df[df["rt_plant_id"] == plant_id]
    # a_["new_col"] = a_.forecast_dt.dt.strftime("%Y") + (a_.forecast_dt.dt.strftime("%W").astype(int) // 2 + 1).astype(str).str.zfill(2)
    a_["new_col"] = a_.forecast_dt.dt.strftime("%Y%m")
    a_ = a_.groupby("new_col").production.max().reset_index()
    x = a_.production.rolling(3, closed="left")
    a_["outlier1"] = x.mean() + 3 * x.std()
    a_.loc[a_["production"] > a_["outlier1"], "production"] = a_["outlier1"]
    a_["outlier2"] = x.mean() - 3 * x.std()
    a_.loc[a_["production"] < a_["outlier2"], "production"] = a_["outlier2"]
    a_.drop(["outlier1", "outlier2"], axis=1, inplace=True)
    return a_
    # ax = axes[1]
    # ax.plot(a_.set_index("new_col")["production"]);
    # ax.set_title("Outlier 1");


In [ ]:
# _remove_local_outliers(df, PLANT_ID).set_index("new_col").plot(figsize=(15,5))

In [6]:
df

,epoch,production,rt_plant_id,forecast_dt,UGRD_80.m.above.ground.SW,UGRD_80.m.above.ground.NW,UGRD_80.m.above.ground.NE,UGRD_80.m.above.ground.SE,VGRD_80.m.above.ground.SW,VGRD_80.m.above.ground.NW,VGRD_80.m.above.ground.NE,VGRD_80.m.above.ground.SE,production_lower,production_upper,is_outlier
0,1548288000,10.0,1503,2019-01-24 03:00:00+03:00,-2.381990,-1.811990,-1.301990,-1.89199,17.66920,15.35920,15.56920,10.63920,0.0,10.0,False
1,1548291600,10.0,1503,2019-01-24 04:00:00+03:00,-3.177610,-2.647610,-2.297610,-3.02761,16.50070,14.29070,14.46070,10.24070,0.0,10.0,False
2,1548295200,9.0,1503,2019-01-24 05:00:00+03:00,-3.593300,-3.043300,-2.573300,-3.40330,15.63730,13.43730,13.17730,9.39729,0.0,10.0,False
3,1548298800,2.7,1503,2019-01-24 06:00:00+03:00,-4.027460,-3.627460,-2.697460,-3.66746,14.39660,13.14660,11.37660,8.86664,0.0,10.0,False
4,1548302400,1.3,1503,2019-01-24 07:00:00+03:00,-4.512970,-4.192970,-2.722970,-3.82297,13.09370,12.96370,9.64367,8.58367,0.0,10.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2650409,1643299200,20.0,2374,2022-01-27 19:00:00+03:00,-0.014146,-0.894145,0.225854,-1.14415,-9.65097,-5.86097,-5.42097,-4.08097,0.0,20.0,False
2650410,1643302800,18.5,2374,2022-01-27 20:00:00+03:00,0.542844,-0.977156,-0.307156,-1.18716,-9.24466,-5.59466,-4.40466,-3.31466,0.0,20.0,False
2650411,1643306400,18.1,2374,2022-01-27 21:00:00+03:00,-0.170332,-1.980330,-0.790332,-1.39033,-9.34740,-5.69740,-3.84740,-3.20740,0.0,20.0,False
2650412,1643310000,13.7,2374,2022-01-27 22:00:00+03:00,-0.388333,-2.448330,-0.778333,-1.53833,-8.65053,-5.44053,-3.42053,-2.99053,0.0,20.0,False


In [ ]:
# a.set_index("forecast_dt").ratio.rolling(30*7*24).mean().plot(figsize=(15,20))
# a.set_index("forecast_dt").ratio.plot(figsize=(15,20))
# a.set_index("forecast_dt").ratio.rolling(30*7*24).max().plot(figsize=(15,20))

# rolling outlier detection
a["ratio"] = a["max_week_production"] / a["cum_max_production"]
r = a.rolling(window=10*7*24).ratio
mps = r.mean() + 3. * r.std() 
a["outlier"] = mps
# a.set_index("forecast_dt")[["ratio", "outlier"]].plot(figsize=(15,20))

In [ ]:
# to find the change in the scale we need to implement the following:
# 0. (not so sure) clip outliers globally
# 1. find local outliers and remove them - en granüler seviyede std çok yüksek outlier atamadım
# 2. find the max production in the month
# 3. find the percentage change for each month



In [ ]:
df.sort_values(["rt_plant_id", "forecast_dt"], inplace=True)
mean_ = df.groupby("rt_plant_id", as_index=False).production.rolling(3, closed="left").mean()
mean_.name = "mean"
std_ = df.groupby("rt_plant_id", as_index=False).production.rolling(3, closed="left").std()
std_.name = "std"

In [ ]:
PLANT_ID = 1488
a__ = df[df["rt_plant_id"] == PLANT_ID]
a__.groupby(a__.forecast_dt.dt.strftime("%Y%m")).production.max().plot(figsize=(15,5))

In [ ]:
a.groupby("new_col").production.max().plot(figsize=(15,5))

In [ ]:
a

In [ ]:
a[a["new_col"] == "20180410"]

In [ ]:
a.groupby("new_col").production.max().pct_change().rolling(2).sum()

In [ ]:
a.groupby("new_col").production.max().pct_change()

In [ ]:
import matplotlib.pyplot as plt

N = 3
fig, axes = plt.subplots(N, figsize=(5*N,5*N))

a_ = a.groupby("new_col").production.max().reset_index()

# original
ax = axes[0]
ax.plot(a_.set_index("new_col")["production"]);
ax.set_title("Original");

# rolling window 3
x = a_.production.rolling(3, closed="left")
a_["outlier1"] = x.mean() + 3 * x.std()
a_.loc[a_["production"] > a_["outlier1"], "production"] = a_["outlier1"]
a_["outlier2"] = x.mean() - 3 * x.std()
a_.loc[a_["production"] < a_["outlier2"], "production"] = a_["outlier2"]
ax = axes[1]
ax.plot(a_.set_index("new_col")["production"]);
ax.set_title("Outlier 1");

# rolling window 
a_ = a.groupby("new_col").production.max().reset_index()
x = a_.production.rolling(2, closed="left")
a_["outlier1"] = x.mean() + 3 * x.std()
a_.loc[a_["production"] > a_["outlier1"], "production"] = a_["outlier1"]
a_["outlier2"] = x.mean() - 3 * x.std()
a_.loc[a_["production"] < a_["outlier2"], "production"] = a_["outlier2"]
ax = axes[2]
ax.plot(a_.set_index("new_col")["production"]);
ax.set_title("Outlier 2");



In [ ]:
a_ = a.groupby("new_col").production.max().reset_index()
x = a_.production.rolling(5, closed="left")
a_["outlier1"] = x.mean() + 3 * x.std()
a_.loc[a_["production"] > a_["outlier1"], "production"] = a_["outlier1"]
a_["outlier2"] = x.mean() - 3 * x.std()
a_.loc[a_["production"] < a_["outlier2"], "production"] = a_["outlier2"]
ax = axes[2]
ax.plot(a_.set_index("new_col")["production"]);
ax.set_title("Outlier 2");


In [ ]:
a_ = a.groupby("new_col").production.max().reset_index()
# a_["pct_change"] = a_["production"].pct_change()
# a_.loc[abs(a_["pct_change"]) < 0.1, "pct_change"] = 0
# a_["pct_change2"] = a_["pct_change"].rolling(2).sum()
x = a_.production.rolling(2, closed="left")
a_["outlier1"] = x.mean() + 3 * x.std()
# a_.loc[a_["production"] > a_["outlier1"], "production"] = a_["outlier1"]
a_["outlier2"] = x.mean() - 3 * x.std()
# a_.loc[a_["production"] < a_["outlier2"], "production"] = a_["outlier2"]
a_.set_index("new_col")["production"].plot(figsize=(15,5))

In [ ]:
pd.options.display.max_rows = 999

In [ ]:
df

In [ ]:
TRAIN_END = pd.Timestamp("2022-01-01", tz='Europe/Istanbul')
VALID_START = TRAIN_END
VALID_END = VALID_START + pd.Timedelta(days=1)
TEST_START = VALID_END
TEST_END = TEST_START + pd.Timedelta(days=1)

In [ ]:
df = df.sort_values(["rt_plant_id", "forecast_dt"])
train_indexes = df[df.forecast_dt < TRAIN_END].index
valid_indexes = df[(df.forecast_dt >= VALID_START) & (df.forecast_dt < VALID_END)].index
test_indexes = df[(df.forecast_dt >= TEST_START) & (df.forecast_dt < TEST_END)].index

In [ ]:
y_test = df.loc[test_indexes, "production"].copy()
df.loc[test_indexes, "production"] = np.nan

In [ ]:
for i in range(25, 48):
    df[f"production_shift_{i}"] = df.groupby("rt_plant_id")["production"].shift(i)

In [ ]:
df.drop(columns=["production_lower", "production_upper", "is_outlier"], inplace=True)

In [ ]:
train_df = df.loc[train_indexes]
valid_df = df.loc[valid_indexes]
test_df = df.loc[test_indexes]

In [ ]:
pd.options.display.max_columns = None

In [ ]:
# df.loc[test_indexes, "set"] = "test"
# df.loc[valid_indexes, "set"] = "valid"
# df.loc[train_indexes, "set"] = "train" 

# df[df["rt_plant_id"] == 672][[col for col in df.columns if not col.startswith(("UGRD", "VGRD"))]].query("set in ['test', 'valid', 'train']").tail(60)

In [ ]:
train_df = train_df.dropna()
y_train = train_df["production"]
X_train = train_df.drop(columns=["epoch", "forecast_dt", "production"])

y_valid = valid_df["production"]
X_valid = valid_df.drop(columns=["epoch", "forecast_dt", "production"])

# y_test = test_df["production"]
X_test = test_df.drop(columns=["epoch", "forecast_dt", "production"])

# for col in ["rt_plant_id", "hour", "dayofweek"]:
for col in ["rt_plant_id"]:
    X_train[col] = X_train[col].astype("category")
    X_valid[col] = X_valid[col].astype("category")
    X_test[col] = X_test[col].astype("category")

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    "num_boost_round": 1000,
    'metric': {'l2', 'l1'},
    # 'num_leaves': 51,
    # 'learning_rate': 0.03,
    # 'feature_fraction': 0.9,
    # 'bagging_fraction': 0.8,
    # 'bagging_freq': 5,
    'verbose': 1,
    "categorical_feature": ["rt_plant_id", "hour", "dayofweek"]
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                callbacks=[lgb.early_stopping(stopping_rounds=20)]
                )


In [ ]:
# print('Saving model...')
# save model to file
# gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
from sklearn.metrics import mean_squared_error
rmse_test = mean_squared_error(y_test, y_pred) ** 0.5
print(f'The RMSE of prediction is: {rmse_test}')

In [ ]:
feature_importances = gbm.feature_importance()
feature_names = gbm.feature_name()
pd.DataFrame([feature_importances, feature_names]).T.sort_values(0, ascending=False).head(20)

In [ ]:
test_df["prediction"] = y_pred
test_df["production"] = y_test
y_valid = gbm.predict(X_valid, num_iteration=gbm.best_iteration)
valid_df["prediction"] = y_valid


In [ ]:
test_df["hour"] = test_df["forecast_dt"].dt.hour
test_df.groupby("hour")[["production", "prediction"]].mean().plot(figsize=(20,10))

In [ ]:
valid_df["hour"] = valid_df["forecast_dt"].dt.hour
valid_df.groupby("hour")[["production", "prediction"]].mean().plot(figsize=(20,10))

In [ ]:
test_df.groupby("rt_plant_id").production.sum().sort_values()

In [ ]:
PLANT_ID = 969

In [ ]:
out_ = pd.concat([valid_df[(valid_df["rt_plant_id"] == PLANT_ID)], test_df[(test_df["rt_plant_id"] == PLANT_ID)]])
out_.set_index("forecast_dt", inplace=True)
out_[["production", "prediction"]].plot(figsize=(20,15))

In [ ]:
out_ = test_df[(test_df["rt_plant_id"] == PLANT_ID)]
out_.set_index("forecast_dt", inplace=True)
out_[["production", "prediction"]].plot(figsize=(20,15))

In [ ]:
out_[["production", "prediction"]].sum()